In [ ]:
import numpy as np
import pandas as pd 

def transform(x):
    x['cp_type'] = x[x['cp_type']=='trt_cp']
    x.pop('cp_type')
    x['cp_dose'] = x['cp_dose'].replace({'D1':-1, 'D2':1})
    x['cp_time'] = x['cp_time'] // 24
    x.join(pd.get_dummies(x['cp_time']))
    x.pop('cp_time')
    return x



In [ ]:
X_all = pd.read_csv('/kaggle/input/lish-moa/train_features.csv')
y_all = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')
whatisit = pd.read_csv('/kaggle/input/lish-moa/train_targets_nonscored.csv')
X = transform(X_all[X_all.columns[1:]])
X_test = pd.read_csv('/kaggle/input/lish-moa/test_features.csv')
X_tezt = transform(X_test[X_test.columns[1:]])

B = np.linalg.pinv(X)
C = y_all[y_all.columns[1:]]
D = np.dot(B,C)
subm = np.dot(X_tezt,D)


In [ ]:
X


# Необходмо подобрать веса для фукнции application_transform(x), таким образом, чтобы все веса в сумме давали 1.0


In [ ]:
eps = 0.000001
def tgfunc(x):
    return np.pi * np.tan(x) / 4.0
def x3func(x):
    return np.power(x, 3.0)
def x5func(x):
    return np.power(x, 5.0)
def application_transform(x):
    x1 = 0.3*tgfunc(x) + 0.3*x3func(x) + 0.3*x5func(x) + 0.1*x 
    if x1 < eps:
        x1 = eps
    elif x1 >= 1:
        x1 = 1 - eps
    return x1
    
submission = pd.read_csv('/kaggle/input/lish-moa/sample_submission.csv')

length = len(submission.columns)-1
df = pd.DataFrame(subm,columns = submission.columns[1:])
df = df.applymap(application_transform)
df['sig_id'] = submission['sig_id']
df = df[submission.columns]
for i in range(len(submission)): 
    if X_test.loc[i,'cp_type']!='trt_cp':
        df.loc[i] = [df.loc[i,'sig_id']]+[0] * length 




In [ ]:
df

In [ ]:
df.to_csv('submission.csv', index=False) #, float_format="%.10f")